In [19]:
import os
import cv2
from ultralytics import YOLO
from tqdm import tqdm  

from utils import ObjectMap as mp
image_folder = 'redfin_images'

model = YOLO("best.pt")

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for image_name in tqdm(image_files, desc="Processing Images"):
    image_path = os.path.join(image_folder, image_name)
    
    image = cv2.imread(image_path)
    
    results = model.predict(image)

    if len(results[0].boxes) == 0:
        os.remove(image_path)
print("Processing complete.")


Processing Images:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Images: 100%|██████████| 2/2 [00:01<00:00,  1.32it/s]

{'Standard-Chair': 0.8889604806900024, 'Standard-Table': 0.8504302501678467}
--------------
{'Painting': 0.9338606595993042, 'Armchair': 0.9041309356689453, 'Side-Table': 0.8427183628082275}
--------------
Processing complete.


In [40]:
import os
import cv2
import pandas as pd
from ultralytics import YOLO
from tqdm import tqdm

model = YOLO("best.pt")
image_folder = 'complete_data/redfin_images'

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

df_results = pd.DataFrame(columns=['Image Path', 'Detected Objects'])

for image_name in tqdm(image_files, desc="Processing Images"):
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)
    results = model.predict(image,conf=0.7,verbose=False)

    objects_confidence = {}

    if len(results[0].boxes) < 2:
        os.remove(image_path)
        continue
    for result in results:
        
        for box in sorted(result.boxes, key=lambda x: x.conf, reverse=True):
            cls = model.names[int(box.cls.item())]  

            if cls not in objects_confidence or box.conf.item() > objects_confidence[cls]:
                objects_confidence[cls] = box.conf.item()

    temp_df = pd.DataFrame({
        'Image Path': [image_name],
        'Detected Objects': [objects_confidence]
    })
    df_results = pd.concat([df_results, temp_df], ignore_index=True)

print(df_results)

df_results.to_pickle("detection_results.pkl")


Processing Images: 100%|██████████| 23339/23339 [47:22<00:00,  8.21it/s] 

             Image Path                                   Detected Objects
0       148731117_9.jpg  {'Table-Lamp': 0.9080426692962646, 'Side-Table...
1        39920005_8.jpg  {'Big-Bed': 0.9165928363800049, 'Sideboard-Cre...
2       35734512_21.jpg  {'Armchair': 0.8945357203483582, 'Ottoman-Chai...
3       174131950_8.jpg  {'Standard-Sofa': 0.9400953650474548, 'Table-L...
4        9945558_46.jpg  {'Big-Bed': 0.9229161143302917, 'Painting': 0....
...                 ...                                                ...
16312   190240742_4.jpg  {'Painting': 0.8033503293991089, 'Armchair': 0...
16313   144018358_1.jpg  {'Standard-Sofa': 0.8952598571777344, 'Armchai...
16314  167623786_18.jpg  {'Painting': 0.945020854473114, 'Sideboard-Cre...
16315   147212133_7.jpg  {'Standard-Chair': 0.8914758563041687, 'Standa...
16316  167635040_16.jpg  {'Side-Table': 0.9002524614334106, 'Mirror': 0...

[16317 rows x 2 columns]


In [42]:
df_results['Detected Objects'][5].keys()


dict_keys(['Armchair', 'Table-Lamp', 'Standard-Chair'])

In [43]:
for i in df_results[:2]:
    print(i)

Image Path
Detected Objects


In [44]:
df_results.head()

,Image Path,Detected Objects
0,148731117_9.jpg,"{'Table-Lamp': 0.9080426692962646, 'Side-Table..."
1,39920005_8.jpg,"{'Big-Bed': 0.9165928363800049, 'Sideboard-Cre..."
2,35734512_21.jpg,"{'Armchair': 0.8945357203483582, 'Ottoman-Chai..."
3,174131950_8.jpg,"{'Standard-Sofa': 0.9400953650474548, 'Table-L..."
4,9945558_46.jpg,"{'Big-Bed': 0.9229161143302917, 'Painting': 0...."


In [47]:
import os
import shutil

source_directory = 'complete_data/redfin_images'

destination_directory = 'Bedrooms'

if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

cnt = 0

for index, row in df_results.iterrows():
    detected_objects = row['Detected Objects']
    image_name = row['Image Path']

    if 'Big-Bed' in detected_objects or 'Twin-Bed' in detected_objects:
        cnt += 1
        source_image_path = os.path.join(source_directory, image_name)

        destination_image_path = os.path.join(destination_directory, image_name)

        shutil.copy(source_image_path, destination_image_path)
        print(f"Image {image_name} copied to {destination_directory}")

print(f"Total images copied: {cnt}")


Image 148731117_9.jpg copied to Beds
Image 39920005_8.jpg copied to Beds
Image 35734512_21.jpg copied to Beds
Image 9945558_46.jpg copied to Beds
Image 28496854_8.jpg copied to Beds
Image 12354445_9.jpg copied to Beds
Image 9968988_20.jpg copied to Beds
Image 102282003_18.jpg copied to Beds
Image 179455171_18.jpg copied to Beds
Image 190345462_20.jpg copied to Beds
Image 144161210_13.jpg copied to Beds
Image 6014942_12.jpg copied to Beds
Image 9974160_14.jpg copied to Beds
Image 148058076_10.jpg copied to Beds
Image 902789_20.jpg copied to Beds
Image 1617326_27.jpg copied to Beds
Image 4524538_10.jpg copied to Beds
Image 45477966_40.jpg copied to Beds
Image 85488081_9.jpg copied to Beds
Image 45477966_39.jpg copied to Beds
Image 1862329_15.jpg copied to Beds
Image 36499501_11.jpg copied to Beds
Image 59144141_16.jpg copied to Beds
Image 21647595_8.jpg copied to Beds
Image 649262_15.jpg copied to Beds
Image 182097378_11.jpg copied to Beds
Image 190328720_14.jpg copied to Beds
Image 1666